In [ ]:
# Central Forces — Leapfrog Integrator (Python version)

This notebook translates the original MATLAB assignment on central‐force motion into Python.  
You will:

* Implement a **velocity verlet integrator** for radial motion with conserved angular momentum \(L\).
* Explore several force laws – inverse‑square, inverse, inverse‑cube, linear (Hooke‑like), and Yukawa.
* Generate **polar plots** of the trajectory and **radius‑vs‑time** graphs with Plotly.
* Answer qualitative questions in the provided markdown prompts.

> **Tip** – Run the notebook cell‑by‑cell. Code is fully parameterised so you can change initial conditions, total time, or `dt` without rewriting functions.


In [ ]:
import numpy as np
import plotly.graph_objects as go

def leapfrog_central(total_time: float,
                     dt: float,
                     r0: float,
                     vr0: float,
                     theta0: float,
                     L: float,
                     m: float,
                     accel_fn,
                     **accel_params):
    """Simulate central‑force motion with the leapfrog integrator.

    Parameters
    ----------
    total_time : float
        End time of the simulation.
    dt : float
        Time‑step (must evenly divide total_time).
    r0, vr0, theta0 : float
        Initial radius, radial velocity, and angle (rad).
    L : float
        Conserved angular momentum.
    m : float
        Particle mass.
    accel_fn : callable
        Function `a_r = accel_fn(r, m=m, k=1, **accel_params)` returning the
        *radial* acceleration **excluding** the centripetal term L²/(m² r³).
    accel_params : dict
        Extra keyword arguments forwarded to *accel_fn*.
    """

    n_steps = int(total_time/dt)
    t = np.linspace(0, total_time, n_steps + 1)

    # Allocate arrays
    r = np.empty_like(t)
    vr = np.empty_like(t)
    ar = np.empty_like(t)

    theta = np.empty_like(t)
    vtheta = np.empty_like(t)
    thetadot = np.empty_like(t)

    # Initial conditions
    r[0], vr[0], theta[0] = r0, vr0, theta0
    thetadot[0] = L / (m * r0**2)
    vtheta[0] = r0 * thetadot[0]
    ar_extra = L**2 / (m**2 * r0**3)    # centripetal term
    ar[0] = accel_fn(r0, m=m, **accel_params) + ar_extra

    # --- Leapfrog integration ---
    for i in range(n_steps):
        # Drift: update radius
        r[i+1] = r[i] + vr[i]* dt+ar[i]*dt*dt*0.5

        # Update theta quantities **after** radius is known
        thetadot[i+1] = L / (m * r[i+1]**2)
        vtheta[i+1] = r[i+1] * thetadot[i+1]
        theta[i+1] = theta[i] + vtheta[i+1] * dt  # leapfrog form

        # Compute acceleration at new position
        ar_extra = L**2 / (m**2 * r[i+1]**3)
        ar[i+1] = accel_fn(r[i+1], m=m, **accel_params) + ar_extra

        # Record full‑step radial velocity for output
        vr[i+1] = vr[i]+ 0.5 * (ar[i]+ar[i+1]) * dt

    return t, r, vr, theta, vtheta, thetadot


In [ ]:
def inv_square_accel(r, k=1.0, m=1.0, **_):
    """Inverse‑square force  F = -k / r²  (Newton/Kepler)."""
    return -k / r**2 / m

def inv_accel(r, k=1.0, m=1.0, **_):
    """Inverse force  F = -k / r  (e.g., 2‑D gravity analogue)."""
    return -k / r / m

def inv_cube_accel(r, k=1.0, m=1.0, **_):
    """Inverse‑cube force  F = -k / r³."""
    return -k / r**3 / m

def linear_accel(r, k=1.0, m=1.0, **_):
    """Linear (Hooke‑like) force  F = -k r."""
    return -k * r / m

def yukawa_accel(r, k=1.0, a=1.0, m=1.0, **_):
    """Yukawa potential  V(r) = -k e^{‑a r} / r  ⇒
    F(r) = -∂V/∂r = -k e^{‑a r} (1/r² + a/r)."""
    return -k * np.exp(-a * r) * (1/r**2 + a/r) / m


In [ ]:
def plot_polar(r, theta, title=''):
    fig = go.Figure()

    fig.add_trace(
        go.Scatterpolar(
            r=r,
            theta=np.degrees(theta),   # Plotly expects degrees
            mode='lines',
            line=dict(width=2)
        )
    )

    fig.update_layout(
        title_text=title,                      # overall title
        polar=dict(
            radialaxis=dict(                   # **radial** axis can have a title
                title=dict(text='Radius r')
            ),
            angularaxis=dict(                  # no title here; rotation options only
                rotation=0,                    # 0° at +x direction
                direction="counterclockwise"
            )
        ),
        width=600,
        height=600,
        showlegend=False
    )

    fig.show()


def plot_radius_time(t, r, title='r(t)'):
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=t, y=r, mode='lines', name='r'))
    fig.update_layout(
        title=title,
        xaxis_title='Time',
        yaxis_title='Radius',
        width=700,
        height=400
    )
    fig.show()


## 1 · Baseline inverse‑square simulation

Initial parameters  
\[
k = 1, \quad m = 1, \quad L = 2, \quad r_0 = L^2 = 4, \quad v_{r0}=0,\quad \theta_0=0,\quad
t_\text{max}=100,\; dt=10^{-4}
\]

Run the cell and inspect the polar trajectory and radius‑vs‑time.  
Afterwards, **edit the markdown immediately below** to answer the first question: *What sort of motion do we have?*

In [ ]:
# --- parameters ---
k, m, L = 1.0, 1.0, 2.0
r0 = L**2
vr0 = 0.0
theta0 = 0.0
t_max = 100.0
dt = 1e-3   # 0.0001

# --- simulate ---
t, r, vr, theta, vtheta, thetadot = leapfrog_central(
    total_time=t_max, dt=dt,
    r0=r0, vr0=vr0, theta0=theta0,
    L=L, m=m,
    accel_fn=inv_square_accel, k=k
)

# --- plots ---
plot_polar(r, theta, title='Inverse‑square baseline (polar)')
plot_radius_time(t, r, title='Inverse‑square baseline  r(t)')


**Your observations:**  
_Double‑click to edit. Describe the type of orbit and any conserved quantities you notice._

## 2 · Experiments

The following cells provide **templates** for each experiment in the assignment.  
Adjust parameters as directed, run, and record your observations in the markdown prompts.


### 2.1 Increase $r_0$ by 2× and time by 10×  
(Keep the inverse‑square force.)

In [ ]:
# 2.1 parameters
r0 = 2 * L**2
t_max = 200.0           # 10× longer
dt = 1e-3

t, r, vr, theta, vtheta, thetadot = leapfrog_central(
    total_time=t_max, dt=dt,
    r0=r0, vr0=0.0, theta0=0.0,
    L=L, m=m,
    accel_fn=inv_square_accel, k=k
)
plot_polar(r, theta, title='Inverse‑square, r0×2, t×10')
plot_radius_time(t, r, title='r(t)  —  r0×2, t×10')


**Observations:**  
_Discuss how the orbit shape and radial oscillation period change._

### 2.2 Inverse force $F=-k/r$  
(a) Baseline parameters ( $r_0=L^2$, $t=100$ )

In [ ]:
t, r, vr, theta, vtheta, thetadot = leapfrog_central(
    total_time=100.0, dt=1e-3,
    r0=L**2, vr0=0.0, theta0=0.0,
    L=L, m=m,
    accel_fn=inv_accel, k=k
)
plot_polar(r, theta, title='Inverse‑first‑power force  (polar)')
plot_radius_time(t, r, title='Inverse‑first‑power  r(t)')


_Describe the motion. Is it closed or precessing?_

**(c) Double $r_0$ ( $r_0=2L^2$ ) with $t=1000$**

In [ ]:
t, r, vr, theta, vtheta, thetadot = leapfrog_central(
    total_time=200.0, dt=1e-3,
    r0=2*2**2, vr0=0.0, theta0=0.0,
    L=L, m=m,
    accel_fn=inv_accel, k=k
)
plot_polar(r, theta, title='Inverse force — r0×2')
plot_radius_time(t, r, title='Inverse force — r0×2  r(t)')


_Notes:_

### 2.3 Inverse‑cube force $F=-k/r^3$ (baseline)

In [ ]:
t, r, vr, theta, vtheta, thetadot = leapfrog_central(
    total_time=100.0, dt=1e-3,
    r0=L**2, vr0=0.0, theta0=0.0,
    L=L, m=m,
    accel_fn=inv_cube_accel, k=k
)
plot_polar(r, theta, title='Inverse‑cube baseline')
plot_radius_time(t, r, title='Inverse‑cube  r(t)')


_Describe the orbit — does it remain bounded?_

### 2.4 Linear force  $F=-k\,r$  
(a) $r_0=L^2,\;L=2$

In [ ]:
t, r, vr, theta, vtheta, thetadot = leapfrog_central(
    total_time=100.0, dt=1e-3,
    r0=L**2, vr0=0.0, theta0=0.0,
    L=2.0, m=1.0,
    accel_fn=linear_accel, k=1.0
)
plot_polar(r, theta, title='Linear force  (L=2)')
plot_radius_time(t, r, title='Linear force  r(t)  (L=2)')


**(b) $r_0=4,\;L=0$**

In [ ]:
t, r, vr, theta, vtheta, thetadot = leapfrog_central(
    total_time=100.0, dt=1e-3,
    r0=4.0, vr0=0.0, theta0=0.0,
    L=0.0, m=1.0,
    accel_fn=linear_accel, k=1.0
)
plot_polar(r, theta, title='Linear force  (L=0)')
plot_radius_time(t, r, title='Linear force  r(t)  (L=0)')


_Explain differences between the two cases._

### 2.5 Yukawa potential  $V(r) = -\dfrac{k e^{-a r}}{r}$

Below we sweep the screening length parameter $a$ over decades (0.001 → 0.01 → 0.1 → 1.0) and plot the resulting orbits.


In [ ]:
a_values = [1e-3, 1e-2, 1e-1, 1.0]
for a in a_values:
    t, r, vr, theta, vtheta, thetadot = leapfrog_central(
        total_time=500.0, dt=1e-3,
        r0=L**2, vr0=0.0, theta0=0.0,
        L=2.0, m=1.0,
        accel_fn=yukawa_accel, k=1.0, a=a
    )
    plot_polar(r, theta, title=f'Yukawa  a={a}')


_For each $a$, record qualitative changes: does the orbit remain closed, precess, or become unbound?_

## 3 · Conclusions

Summarise the key dynamical behaviours observed across the different force laws and parameter variations.  
Which forces yield closed orbits? Where do we observe precession or runaway trajectories?  
Link your explanations to conservation of energy and angular momentum.


# 